In [1]:
from equation.option_pricing import BlackScholes
from optimize.option_princing import BlackScholeOptimizer
from method.nn import MLP, ResNet
from method.hnn import HybridCQN
from method.qnn import QuantumNeuralNetwork, CorrelatorQuantumNeuralNetwork
from utils.adapters import ResNetFeatures, MLPFeatures

from tqdm import tqdm
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import torch as tc
import matplotlib.pyplot as plt

2025-11-05 14:41:05.625153: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 14:41:05.656197: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 14:41:06.283585: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import torch
print("torch:", torch.__version__)
print("CUDA runtime:", torch.version.cuda)
print("cuda available?", torch.cuda.is_available())
print("device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)


torch: 2.8.0+cu128
CUDA runtime: 12.8
cuda available? True
device name: NVIDIA GeForce GTX 1650


In [3]:
# 1) escolha do device (uma string é suficiente para nossas classes)
device = "cpu"   # "cuda" para forçar GPU, "cpu" para CPU, "auto" escolhe GPU se houver
dtype  = tc.float32

# 2) dados (sem mudanças)
bse = BlackScholes(eps=1e-10)
data = bse.generate_data()
data_teste = bse.generate_data(seed=42)

epochs = 10
lr = 2e-2



In [4]:
# 3) MLP clássico (ajuste os nomes dos args: era hidden/blocks)
model = MLP(hidden=4, blocks=5, activation=nn.Tanh(),
            device=device)

opt = BlackScholeOptimizer(data, model,
                           epochs=epochs, lr=lr,
                           device=device)

loss = opt.train(return_loss=True)

Trainning: 100%|██████████| 10/10 [00:00<00:00, 56.60it/s]


In [5]:
# 4) QNN + híbrido
qnn = QuantumNeuralNetwork(n_qubits=5, n_layers=4,
                           device=device, dtype=dtype)

hybrid = HybridCQN(classical_pre=model, qnn_block=qnn, classical_post=None,
                   input_dim=2, output_dim=1,
                   device=device, dtype=dtype)

qpinn_opt = BlackScholeOptimizer(data, hybrid,
                                 epochs=epochs, lr=lr,
                                 device=device, dtype=dtype)

loss_qpinn = qpinn_opt.train(return_loss=True)

Trainning: 100%|██████████| 10/10 [00:02<00:00,  4.53it/s]


In [ ]:
for name, param in hybrid.named_parameters():
    print(name, param.device)


In [ ]:
bse = BlackScholes(eps=1e-10)
data = bse.generate_data()
data_teste = bse.generate_data(seed=42)
epochs = 1000
lr = 2e-2

In [ ]:
model = MLP(neurons=4, M=5, activation=nn.Tanh())
opt = BlackScholeOptimizer(data, model, epochs=epochs, lr=lr)
loss = opt.train(return_loss=True)

In [ ]:
res_model = ResNet(hidden=4, blocks=5, activation=nn.Tanh())
res_opt = BlackScholeOptimizer(data, res_model, epochs=epochs, lr=lr)
loss_ress = res_opt.train(return_loss=True)

In [ ]:
qnn = QuantumNeuralNetwork(n_qubits=5, n_layers=4)
qpinn_opt = BlackScholeOptimizer(data, HybridCQN(classical_pre=None, qnn_block=qnn, classical_post=None), epochs=epochs, lr=lr)
loss_qpinn= qpinn_opt.train(return_loss=True)

In [9]:
cqnn = CorrelatorQuantumNeuralNetwork(n_qubits=3, n_layers=4, k=2, n_vertex=5)
hybrid = HybridCQN(classical_pre=model, qnn_block=cqnn, classical_post=None)
cqpinn_opt = BlackScholeOptimizer(data, hybrid, epochs=epochs, lr=lr)
loss_cqpinn= cqpinn_opt.train(return_loss=True)

Trainning:   0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but got mat1 is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA_addmm)

In [10]:
for name, param in hybrid.named_parameters():
    print(name, param.device)


pre.hidden_layers.0.weight cuda:0
pre.hidden_layers.0.bias cuda:0
pre.hidden_layers.1.weight cuda:0
pre.hidden_layers.1.bias cuda:0
pre.hidden_layers.2.weight cuda:0
pre.hidden_layers.2.bias cuda:0
pre.hidden_layers.3.weight cuda:0
pre.hidden_layers.3.bias cuda:0
pre.hidden_layers.4.weight cuda:0
pre.hidden_layers.4.bias cuda:0
pre.output_layer.weight cuda:0
pre.output_layer.bias cuda:0
qnn.q_layer.weights cuda:0
to_qubits.weight cuda:0
to_qubits.bias cuda:0
q_out.weight cuda:0
q_out.bias cuda:0


In [ ]:
cqnn_sem = CorrelatorQuantumNeuralNetwork(n_qubits=3, n_layers=4, k=2, n_vertex=5, nonlinear=None)
cqpinn_opt_sem = BlackScholeOptimizer(data, HybridCQN(classical_pre=None, qnn_block=cqnn_sem, classical_post=None), epochs=epochs, lr=lr)
loss_cqpinn_sem= cqpinn_opt_sem.train(return_loss=True)

In [ ]:
plt.plot(loss['Total'], lw=2.5,label='PINN - MLP')
#plt.plot(loss_ress['Total'], lw=2.5,label='PINN - ResNet')
plt.plot(loss_qpinn['Total'], lw=2.5,label='QPINN')
plt.plot(loss_cqpinn['Total'], lw=2.5,label='CQPINN')
plt.plot(loss_cqpinn_sem['Total'], lw=2.5,label='CQPINN - Without') 
#plt.plot(loss_hpinn['Total'], lw=2.5,label='HPINN - MLP')
#plt.plot(loss_hpinn_res['Total'], lw=2.5,label='HPINN - ResNet')
plt.yscale('log')
plt.xscale('log')
plt.legend(fontsize=12)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)

plt.grid(True)
plt.show()

In [ ]:
hpinn_res_opt = BlackScholeOptimizer(data, HybridCQN(classical_pre=ResNetFeatures(ResNet(hidden=4, blocks=1, activation=nn.Tanh())), qnn_block=qnn, classical_post=None), epochs=epochs, lr=lr)
_, loss_hpinn_res = hpinn_res_opt.train(return_loss=True)

In [ ]:

hpinn_opt = BlackScholeOptimizer(data, HybridCQN(classical_pre=MLPFeatures(MLP(neurons=4, M=1, activation=nn.Tanh())), qnn_block=qnn, classical_post=None), epochs=epochs, lr=lr)
_, loss_hpinn = hpinn_opt.train(return_loss=True)

In [ ]:
opt.num_params, res_opt.num_params, qpinn_opt.num_params, hpinn_res_opt.num_params, hpinn_opt.num_params

In [ ]:
# Dictionary to store results for each method
results = {
    'PINN-MLP': {'losses': [], 'best_loss': float('inf'), 'best_history': None},
    'PINN-ResNet': {'losses': [], 'best_loss': float('inf'), 'best_history': None},
    'QPINN': {'losses': [], 'best_loss': float('inf'), 'best_history': None},
    'HPINN-MLP': {'losses': [], 'best_loss': float('inf'), 'best_history': None},
    'HPINN-ResNet': {'losses': [], 'best_loss': float('inf'), 'best_history': None}
}

n_iterations = 10
print(f"Training each model {n_iterations} times...")

for i in tqdm(range(n_iterations)):
    n_iterations = 10
    print(f"Training each model {i}...")
    # PINN-MLP
    model = MLP(neurons=4, M=5, activation=nn.Tanh())
    opt = BlackScholeOptimizer(data, model, epochs=epochs, lr=lr)
    _, current_loss = opt.train(return_loss=True)
    avg_loss = np.mean(current_loss['Total'][-100:])
    results['PINN-MLP']['losses'].append(current_loss['Total'])
    if avg_loss < results['PINN-MLP']['best_loss']:
        results['PINN-MLP']['best_loss'] = avg_loss
        results['PINN-MLP']['best_history'] = current_loss['Total']
    
    # PINN-ResNet
    res_model = ResNet(hidden=4, blocks=5, activation=nn.Tanh())
    res_opt = BlackScholeOptimizer(data, res_model, epochs=epochs, lr=lr)
    _, current_loss = res_opt.train(return_loss=True)
    avg_loss = np.mean(current_loss['Total'][-100:])
    results['PINN-ResNet']['losses'].append(current_loss['Total'])
    if avg_loss < results['PINN-ResNet']['best_loss']:
        results['PINN-ResNet']['best_loss'] = avg_loss
        results['PINN-ResNet']['best_history'] = current_loss['Total']
    
    # QPINN
    qnn = QuantumNeuralNetwork(n_qubits=4, n_layers=4)
    qpinn_opt = BlackScholeOptimizer(data, HybridCQN(classical_pre=None, qnn_block=qnn, classical_post=None), epochs=epochs, lr=lr)
    _, current_loss = qpinn_opt.train(return_loss=True)
    avg_loss = np.mean(current_loss['Total'][-100:])
    results['QPINN']['losses'].append(current_loss['Total'])
    if avg_loss < results['QPINN']['best_loss']:
        results['QPINN']['best_loss'] = avg_loss
        results['QPINN']['best_history'] = current_loss['Total']
    
    # HPINN-ResNet
    qnn = QuantumNeuralNetwork(n_qubits=4, n_layers=3)
    hpinn_res_opt = BlackScholeOptimizer(data, HybridCQN(classical_pre=ResNetFeatures(ResNet(hidden=4, blocks=2, activation=nn.Tanh())), 
                                                         qnn_block=qnn, classical_post=None), epochs=epochs, lr=lr)
    _, current_loss = hpinn_res_opt.train(return_loss=True)
    avg_loss = np.mean(current_loss['Total'][-100:])
    results['HPINN-ResNet']['losses'].append(current_loss['Total'])
    if avg_loss < results['HPINN-ResNet']['best_loss']:
        results['HPINN-ResNet']['best_loss'] = avg_loss
        results['HPINN-ResNet']['best_history'] = current_loss['Total']
    
    # HPINN-MLP
    hpinn_opt = BlackScholeOptimizer(data, HybridCQN(classical_pre=MLPFeatures(MLP(neurons=4, M=1, activation=nn.Tanh())), 
                                                     qnn_block=qnn, classical_post=None), epochs=epochs, lr=lr)
    _, current_loss = hpinn_opt.train(return_loss=True)
    avg_loss = np.mean(current_loss['Total'][-100:])
    results['HPINN-MLP']['losses'].append(current_loss['Total'])
    if avg_loss < results['HPINN-MLP']['best_loss']:
        results['HPINN-MLP']['best_loss'] = avg_loss
        results['HPINN-MLP']['best_history'] = current_loss['Total']



In [ ]:
# Plot best losses
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for method in results:
    plt.plot(results[method]['best_history'], lw=2.5, label=f'{method}\n(Best={results[method]["best_history"][-1]:.2e})')
plt.yscale('log')
plt.xscale('log')
plt.legend(fontsize=10)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Best Loss History', fontsize=14)
plt.grid(True)

# Plot average losses with percentile bands
plt.subplot(1, 2, 2)
for method in results:
    losses_array = np.array(results[method]['losses'])
    median = np.median(losses_array, axis=0)
    q25 = np.percentile(losses_array, 25, axis=0)
    q75 = np.percentile(losses_array, 75, axis=0)
    
    plt.plot(median, lw=2.5, label=method)
    plt.fill_between(range(len(median)), q25, q75, alpha=0.2)

plt.yscale('log')
plt.xscale('log')
plt.legend(fontsize=10)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Median Loss with 25-75 Percentile Band', fontsize=14)
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
param_num =opt.num_params, res_opt.num_params, qpinn_opt.num_params, hpinn_res_opt.num_params, hpinn_opt.num_params

In [ ]:
param_num[0]

In [ ]:
teste = []
for i, method in enumerate(results):
    teste.append(results[method]["best_loss"]/param_num[i])
    print(f"{method}:", results[method]["best_loss"]/param_num[i], results[method]["best_history"][-1]/param_num[i])

print(teste/min(teste))

In [ ]:
# Print statistical summary
print("\nStatistical Summary (based on final 100 epochs average):")
for method in results:
    final_losses = np.array([np.mean(loss[-100:]) for loss in results[method]['losses']])
    q25, median, q75 = np.percentile(final_losses, [25, 50, 75])
    iqr = q75 - q25
    print(f"\n{method}:")
    print(f"Median: {median:.2e}")
    print(f"Q25-Q75: [{q25:.2e} - {q75:.2e}]")
    print(f"IQR: {iqr:.2e}")
    print(f"Stability (IQR/Median): {(iqr/median):.3f}")